In [ ]:
%pip install ipympl -q
%pip install numpy -q
%pip install wheel

%pip install keras -q
%pip install keras-cv -q

%pip install matplotlib -q

In [1]:
location_to_saved_model: str = '/home/spectre/environment_model.keras'

In [2]:
import tensorflow

gpus = tensorflow.config.list_physical_devices(
    'GPU'
)

2024-01-23 19:09:38.043331: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-23 19:09:38.195312: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 19:09:38.195422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 19:09:38.217919: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 19:09:38.274830: I tensorflow/core/platform/cpu_feature_guar

In [3]:
nvidia_maximum_limit_for_gpu_memory: int = 5950

if gpus:
    try:
        for gpu in gpus:
            tensorflow.config.set_logical_device_configuration(
                gpu, 
                [
                    tensorflow.config.LogicalDeviceConfiguration(
                        memory_limit=nvidia_maximum_limit_for_gpu_memory
                    )
                ]
            )
    except RuntimeError as e:
        print(e)

location_to_dataset: str = '/tmp/dataset'
name_of_location: str = 'LocationDataset'

In [ ]:
from os.path import isdir
from shutil import rmtree

if isdir(
    location_to_dataset
):  
    rmtree(
        location_to_dataset
    )

In [ ]:
if isdir(
    name_of_location
):
    rmtree(
        name_of_location
    )

In [1]:
!git clone https://github.com/TargetSpectre/Location-Identification-DataSet.git {name_of_location} -q

!mv ./{name_of_location}/DataSet {location_to_dataset}
!ls {location_to_dataset}

Anubis	Default  Dust-II  Mirage  Vertigo


# 

In [4]:
from os import listdir
from os.path import join

labels: list = list()
found_dirs = listdir(
    location_to_dataset
)

for idx in range(
    len(
        found_dirs
    )
):
    selected = found_dirs[idx]
    labels.append(
        selected
    )

    found_dirs[idx] = join(
        location_to_dataset, 
        selected
    )

labels.sort()

print(
    labels
)

['Anubis', 'Default', 'Dust-II', 'Mirage', 'Vertigo']


In [5]:
from random import SystemRandom
from keras.utils import image_dataset_from_directory

In [6]:
batch_size: int = 2

vision_width: int = 960
vision_height: int = 540

maximum_value: int = 65535

training_dataset = image_dataset_from_directory(
    location_to_dataset,
    labels                  = 'inferred',
    label_mode              = 'categorical',
    color_mode              = 'rgb',
    batch_size              = batch_size,
    image_size              = (
        vision_height, 
        vision_width
    ),
    shuffle                 = True,
    crop_to_aspect_ratio    = True,
    subset                  = 'training',
    validation_split        = 0.25,
    seed                    = SystemRandom().randint(
        1, 
        maximum_value
    )
)

validation_dataset = image_dataset_from_directory(
    location_to_dataset,
    labels                  = 'inferred',
    label_mode              = 'categorical',
    color_mode              = 'rgb',
    batch_size              = batch_size,
    image_size              = (
        vision_height, 
        vision_width
    ),
    shuffle                 = True,
    crop_to_aspect_ratio    = True,
    subset                  = 'validation',
    validation_split        = 0.25,
    seed                    = SystemRandom().randint(
        1, 
        maximum_value
    )
)

Found 12300 files belonging to 5 classes.
Using 9225 files for training.


2024-01-23 19:09:54.053568: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-23 19:09:54.053656: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-23 19:09:54.053673: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-23 19:09:54.375679: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-23 19:09:54.375708: I tensorflow/core/common_runtime/gpu/gpu

Found 12300 files belonging to 5 classes.
Using 3075 files for validation.


In [7]:
from keras import layers
from keras.models import Model

In [26]:
input_layer = layers.Input(
    shape=(
        vision_height, 
        vision_width, 
        3
    )
)

input_layer = layers.Rescaling(
    1./255
)(
    input_layer
)

initial_filter_layer_size: int = 4

middle_layers = layers.Conv2D(
    initial_filter_layer_size, 
    kernel_size = (
        2, 
        2
    ), 
    activation  = 'relu', 
    padding     = 'same'
)(input_layer)

middle_layers = layers.Conv2D(
    initial_filter_layer_size, 
    kernel_size = (
        2, 
        2
    ), 
    activation  = 'relu', 
    padding     = 'same'
)(middle_layers)

middle_layers = layers.Conv2D(
    initial_filter_layer_size, 
    kernel_size = (
        2, 
        2
    ), 
    activation  = 'relu', 
    padding     = 'same'
)(middle_layers)

middle_layers = layers.Conv2D(
    initial_filter_layer_size, 
    kernel_size = (
        2, 
        2
    ), 
    activation  = 'relu', 
    padding     = 'same'
)(middle_layers)

middle_layers = layers.BatchNormalization()(
    middle_layers
)

middle_layers = layers.MaxPooling2D(
    pool_size   = 2, 
    strides     = 2, 
    padding     = 'same'
)(middle_layers)

In [27]:
first_filter_layer_size: int = 8

middle_layers = layers.Conv2D(
    first_filter_layer_size, 
    kernel_size = (
        2, 
        2
    ), 
    activation  = 'relu', 
    padding     = 'same'
)(middle_layers)

middle_layers = layers.Conv2D(
    first_filter_layer_size, 
    kernel_size = (
        2, 
        2
    ), 
    activation  = 'relu', 
    padding     = 'same'
)(middle_layers)

middle_layers = layers.Conv2D(
    first_filter_layer_size, 
    kernel_size = (
        2, 
        2
    ), 
    activation  = 'relu', 
    padding     = 'same'
)(middle_layers)

middle_layers = layers.Conv2D(
    first_filter_layer_size, 
    kernel_size = (
        2, 
        2
    ), 
    activation  = 'relu', 
    padding     = 'same'
)(middle_layers)

middle_layers = layers.BatchNormalization()(
    middle_layers
)

middle_layers = layers.MaxPooling2D(
    pool_size   = 2, 
    strides     = 2, 
    padding     = 'same'
)(middle_layers)


In [28]:
second_filter_layer_size: int = 16

middle_layers = layers.Conv2D(
    second_filter_layer_size, 
    kernel_size=(2, 2), 
    activation='relu', 
    padding='same'
)(middle_layers)

middle_layers = layers.Conv2D(
    second_filter_layer_size, 
    kernel_size=(2, 2), 
    activation='relu', 
    padding='same'
)(middle_layers)

middle_layers = layers.Conv2D(
    second_filter_layer_size, 
    kernel_size=(2, 2), 
    activation='relu', 
    padding='same'
)(middle_layers)

middle_layers = layers.Conv2D(
    second_filter_layer_size, 
    kernel_size=(2, 2), 
    activation='relu', 
    padding='same'
)(middle_layers)

middle_layers = layers.BatchNormalization()(
    middle_layers
)

middle_layers = layers.MaxPooling2D(
    pool_size=2, 
    strides=2, 
    padding='same'
)(middle_layers)

In [29]:
third_filter_layer_size: int = 32

middle_layers = layers.Conv2D(
    third_filter_layer_size, 
    kernel_size=(2, 2), 
    activation='relu', 
    padding='same'
)(middle_layers)

middle_layers = layers.Conv2D(
    third_filter_layer_size, 
    kernel_size=(2, 2), 
    activation='relu', 
    padding='same'
)(middle_layers)

middle_layers = layers.Conv2D(
    third_filter_layer_size, 
    kernel_size=(2, 2), 
    activation='relu', 
    padding='same'
)(middle_layers)

middle_layers = layers.Conv2D(
    third_filter_layer_size, 
    kernel_size=(2, 2), 
    activation='relu', 
    padding='same'
)(middle_layers)

middle_layers = layers.BatchNormalization()(
    middle_layers
)

middle_layers = layers.MaxPooling2D(
    pool_size=2, 
    strides=2, 
    padding='same'
)(middle_layers)

In [30]:
four_filter_layer_size: int = 64

middle_layers = layers.Conv2D(
    four_filter_layer_size, 
    kernel_size=(2, 2), 
    activation='relu', 
    padding='same'
)(middle_layers)

middle_layers = layers.Conv2D(
    four_filter_layer_size, 
    kernel_size=(2, 2), 
    activation='relu', 
    padding='same'
)(middle_layers)

middle_layers = layers.Conv2D(
    four_filter_layer_size, 
    kernel_size=(2, 2), 
    activation='relu', 
    padding='same'
)(middle_layers)

middle_layers = layers.Conv2D(
    four_filter_layer_size, 
    kernel_size=(2, 2), 
    activation='relu', 
    padding='same'
)(middle_layers)

middle_layers = layers.BatchNormalization()(
    middle_layers
)

middle_layers = layers.MaxPooling2D(
    pool_size=2, 
    strides=2, 
    padding='same'
)(middle_layers)


In [31]:
fifth_filter_layer_size: int = 96

middle_layers = layers.Conv2D(
    fifth_filter_layer_size, 
    kernel_size=(2, 2), 
    activation='relu', 
    padding='same'
)(middle_layers)

middle_layers = layers.Conv2D(
    fifth_filter_layer_size, 
    kernel_size=(2, 2), 
    activation='relu', 
    padding='same'
)(middle_layers)

middle_layers = layers.BatchNormalization()(
    middle_layers
)

middle_layers = layers.MaxPooling2D(
    pool_size=2, 
    strides=2, 
    padding='same'
)(middle_layers)

In [32]:
middle_layers = layers.Flatten()(
    middle_layers
)

In [33]:
maximum_number_of_categories: int = len(
    labels
)

output_layer = layers.Dense(
    1024, 
    activation='relu'
)(
    middle_layers
)

output_layer = layers.Dense(
    512, 
    activation='relu'
)(
    output_layer
)

output_layer = layers.Dense(
    maximum_number_of_categories
)(
    output_layer
)


In [34]:
model = Model(
    inputs=input_layer, 
    outputs=output_layer, 
    name='location_identifier'
)

model.summary()

Model: "location_identifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 540, 960, 3)]     0         
                                                                 
 conv2d_44 (Conv2D)          (None, 540, 960, 4)       52        
                                                                 
 conv2d_45 (Conv2D)          (None, 540, 960, 4)       68        
                                                                 
 conv2d_46 (Conv2D)          (None, 540, 960, 4)       68        
                                                                 
 conv2d_47 (Conv2D)          (None, 540, 960, 4)       68        
                                                                 
 batch_normalization_12 (Ba  (None, 540, 960, 4)       16        
 tchNormalization)                                               
                                               

In [35]:
model.compile(
    optimizer   = 'Adam',
    loss        = 'categorical_crossentropy',
    metrics     = ['accuracy']
)

In [37]:
history = model.fit(
    training_dataset, 
    epochs              = 10, 
    validation_data     = validation_dataset,
    workers             = 12,
    callbacks           = callbacks
)

Epoch 1/10


2024-01-23 19:12:43.023225: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-23 19:12:43.203966: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-23 19:12:45.716121: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f3848d1a910 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-23 19:12:45.716173: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2024-01-23 19:12:45.739201: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1706033565.880860   15776 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4613/4613 [==============================] - 190s 38ms/step - loss: 9.3758 - accuracy: 0.3451 - val_loss: 9.8280 - val_accuracy: 0.3567
Epoch 2/10
4613/4613 [==============================] - 176s 38ms/step - loss: 9.4556 - accuracy: 0.3454 - val_loss: 9.8438 - val_accuracy: 0.3571
Epoch 3/10
4613/4613 [==============================] - 178s 39ms/step - loss: 9.4591 - accuracy: 0.3464 - val_loss: 9.8333 - val_accuracy: 0.3567
Epoch 4/10
4613/4613 [==============================] - 175s 38ms/step - loss: 9.3857 - accuracy: 0.3459 - val_loss: 9.8280 - val_accuracy: 0.3564
Epoch 5/10
4613/4613 [==============================] - 177s 38ms/step - loss: 9.4311 - accuracy: 0.3447 - val_loss: 9.8542 - val_accuracy: 0.3577
Epoch 6/10
4613/4613 [==============================] - 173s 37ms/step - loss: 9.4276 - accuracy: 0.3442 - val_loss: 9.8333 - val_accuracy: 0.3571
Epoch 7/10
 973/4613 [=====>........................] - ETA: 2:00 - loss: 9.3180 - accuracy: 0.3541

KeyboardInterrupt: 

In [39]:
model.save(
    location_to_saved_model,
    overwrite = True
)